In [1]:
# Notebook 출력설정
# 주요 라이브러리 임포트

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

%matplotlib inline
import matplotlib.pylab as plt
plt.rcParams["figure.figsize"] = (15,5)
plt.rcParams['lines.linewidth'] = 1
plt.rcParams['axes.grid'] = True
import seaborn as sns

In [2]:
DIR_DATASET = "C:/Users/0stix/Datasets/"
NAME_PROJECT = '2203-kaggle-tps2203'

In [3]:
df_train = pd.read_csv(DIR_DATASET + NAME_PROJECT + '/train.csv', index_col='row_id')
df_test = pd.read_csv(DIR_DATASET + NAME_PROJECT + '/test.csv', index_col='row_id')

len_train = len(df_train)
df_all = pd.concat([df_train, df_test], axis=0)
target_ = 'congestion'

In [4]:
df_v0 = pd.DataFrame()
df_v0[target_] = df_all[target_]
df_v0['drop'] = False

df_v0['dt'] = pd.to_datetime(df_all['time'])

# df_v0['is_hd'] |= df_v0['dt'].dt.date.astype(str).str.contains('1991-05-27|1991-07-04|1991-09-02').astype('int')
df_v0['drop'] |= df_v0['dt'].dt.date.astype(str).str.contains('1991-05-27|1991-07-04|1991-09-02')
# df_v0[df_v0['is_hd'] == 1]['drop'] = 1
# df_v0.drop('is_hd', axis=1, inplace=True)

df_v0['dt_m'] = df_v0['dt'].dt.month
df_v0['dt_d'] = df_v0['dt'].dt.dayofyear
df_v0['dt_am'] = (6 < df_v0['dt'].dt.hour) & (df_v0['dt'].dt.hour < 12)
df_v0['dt_wd'] = df_v0['dt'].dt.weekday
df_v0['dt_t'] = (df_v0['dt'].dt.hour - 12) * 3 + (df_v0['dt'].dt.minute // 20)

df_v0['drop'] |= df_v0['dt_m'] < 5
df_v0['drop'] |= 3 < df_v0['dt_wd'] 

df_v0.drop('dt', axis=1, inplace=True)

df_v0['x'] = df_all['x']
df_v0['y'] = df_all['y']
df_v0['direction'] = df_all['direction']


df_v0['xydirday'] = df_all['x'].astype(str) + df_all['y'].astype(str) + df_all['direction'] + df_v0['dt_d'].astype(str)
df_v0['xydir'] = df_all['x'].astype(str) + df_all['y'].astype(str) + df_all['direction']
df_v0['all'] = df_v0['xydir'] + df_v0['dt_t'].astype(str)
df_v0['drop'] |= df_v0['dt_t'] < 0

# mapper_avg = df_v0[['all',target_]].groupby(['all']).median().to_dict()[target_]
# df_v0['avg'] = df_v0['all'].map(mapper_avg)

# train['avg']= train['all'].map(mapper_avg)
# test['avg']= test['all'].map(mapper_avg)
# train= train[train.time >=0]

# df_v0.head()

In [5]:
df_v0_tr = df_v0[:len_train]
df_v0_te = df_v0[len_train:]

df_v0_tr.drop((df_v0_tr[df_v0_tr['drop'] == True]).index, inplace=True)
df_v0_tr.drop('drop', axis=1, inplace=True)

df_v0_te.drop((df_v0_te[df_v0_te['drop'] == True]).index, inplace=True)
df_v0_te.drop('drop', axis=1, inplace=True)

mapper_avg = df_v0_tr[['all',target_]].groupby(['all']).median().to_dict()[target_]
df_v0_tr['avg'] = df_v0_tr['all'].map(mapper_avg)
df_v0_te['avg'] = df_v0_te['all'].map(mapper_avg)

df_v0_tr.head()

,congestion,dt_m,dt_d,dt_am,dt_wd,dt_t,x,y,direction,xydirday,xydir,all,avg
row_id,,,,,,,,,,,,,
142220,27.0,5,121,False,2,0,0,0,EB,00EB121,00EB,00EB0,50.0
142221,24.0,5,121,False,2,0,0,0,NB,00NB121,00NB,00NB0,35.0
142222,52.0,5,121,False,2,0,0,0,SB,00SB121,00SB,00SB0,55.0
142223,27.0,5,121,False,2,0,0,1,EB,01EB121,01EB,01EB0,26.0
142224,72.0,5,121,False,2,0,0,1,NB,01NB121,01NB,01NB0,72.0


In [6]:
df_v0_tr.shape, df_v0_te.shape, df_v0.shape

((193700, 13), (2340, 13), (851175, 13))

In [7]:
from pycaret.regression import setup as pc_setup

reg = pc_setup(data = df_v0_tr,
            target = target_,
            session_id=999,
            data_split_shuffle = True, 
            create_clusters = False,
            fold_strategy = 'groupkfold',
            fold_groups = 'dt_wd',
            use_gpu = False,
            silent = True,
            fold=4,
            ignore_features = ['all','dt_d','xydirday'],
            n_jobs = -1)

,Description,Value
0,session_id,999
1,Target,congestion
2,Original Data,"(193700, 13)"
3,Missing Values,0
4,Numeric Features,2
5,Categorical Features,7
6,Ordinal Features,0
7,High Cardinality Features,0
8,High Cardinality Method,None
9,Transformed Train Set,"(135589, 92)"


In [8]:
from pycaret.regression import compare_models

top5 = compare_models(sort='MAE', 
                      n_select=4, 
                      exclude=[
                          'lar', 
                          'rf', 
                          'et', 
                          # 'gbr', 
                          # 'xgboost'
                      ])

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,5.4533,58.9958,7.6808,0.7899,0.1971,0.1287,3.7825
lightgbm,Light Gradient Boosting Machine,5.4700,59.6567,7.7237,0.7875,0.1982,0.1292,0.3250
xgboost,Extreme Gradient Boosting,5.4825,59.6317,7.7221,0.7876,0.1980,0.1294,4.0475
huber,Huber Regressor,5.4833,61.3997,7.8358,0.7813,0.1997,0.1273,6.6100
gbr,Gradient Boosting Regressor,5.5114,60.7048,7.7913,0.7838,0.1996,0.1301,4.9125
lasso,Lasso Regression,5.5210,61.8099,7.8619,0.7798,0.2010,0.1293,0.6850
en,Elastic Net,5.5210,61.8098,7.8619,0.7798,0.2010,0.1293,0.1400
omp,Orthogonal Matching Pursuit,5.5239,61.1793,7.8217,0.7821,0.2002,0.1300,0.1075
br,Bayesian Ridge,5.5272,61.0999,7.8166,0.7824,0.2001,0.1302,1.2250
ridge,Ridge Regression,5.5338,61.1029,7.8168,0.7824,0.2001,0.1304,0.6675


In [75]:
top5[0] = model

In [76]:
from pycaret.regression import blend_models

blender = blend_models(top5)

In [77]:
from pycaret.regression import finalize_model

final = finalize_model(blender)

In [78]:
df_sub = pd.read_csv(DIR_DATASET + NAME_PROJECT + '/sample_submission.csv', index_col='row_id')

In [79]:
from pycaret.regression import predict_model

df_sub[target_] = (predict_model(final, data=df_v0_te)['Label']).round().copy()
# df_sub[target_] = predict_model(final, data=df_v0_te)
# pred_ = predict_model(final, data=df_v0_te)

sep = df_v0_tr[(df_v0_tr['dt_d'] >= 246) & (df_v0_tr['dt_t'] >= 0)]
lower = sep.groupby(['dt_t', 'x', 'y', 'direction']).congestion.quantile(0.15).values
upper = sep.groupby(['dt_t', 'x', 'y', 'direction']).congestion.quantile(0.7).values

df_sub[target_] = df_sub[target_].clip(lower, upper)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Voting Regressor,0,0,0,0,0,0


In [80]:
import datetime

now = datetime.datetime.now()
str_datetime = now.strftime("%y%m%d_%H%M%S")
# df_sub.to_csv(DIR_DATASET+'submission-'+NAME_PROJECT+'-'+str_datetime+'.csv', index = 0)
df_sub.to_csv(DIR_DATASET+'submission-'+NAME_PROJECT+'-'+str_datetime+'.csv')

In [41]:
df_v1 = df_v0.drop((df_v1[df_v1['drop'] == True]).index)
df_v1.head()

,congestion,drop,dt_m,dt_d,dt_am,dt_wd,dt_t,x,y,direction,xydirday,xydir,all
row_id,,,,,,,,,,,,,
142220,27.0,False,5,121,False,2,0,0,0,EB,00EB121,00EB,00EB0
142221,24.0,False,5,121,False,2,0,0,0,NB,00NB121,00NB,00NB0
142222,52.0,False,5,121,False,2,0,0,0,SB,00SB121,00SB,00SB0
142223,27.0,False,5,121,False,2,0,0,1,EB,01EB121,01EB,01EB0
142224,72.0,False,5,121,False,2,0,0,1,NB,01NB121,01NB,01NB0


In [57]:
df_v2 = pd.DataFrame()
df_v2[target_] = df_v1[target_]
df_v2 = pd.concat([df_v2, pd.get_dummies(df_v1['dt_m'].astype('category').cat.codes, prefix='dt_m')], axis=1)
df_v2['dt_am'] = df_v1['dt_am'].astype(int)
df_v2 = pd.concat([df_v2, pd.get_dummies(df_v1['dt_wd'].astype('category').cat.codes, prefix='dt_wd')], axis=1)
df_v2['dt_t'] = df_v1['dt_t']
df_v2 = pd.concat([df_v2, pd.get_dummies(df_v1['x'].astype('category').cat.codes, prefix='x')], axis=1)
df_v2 = pd.concat([df_v2, pd.get_dummies(df_v1['y'].astype('category').cat.codes, prefix='y')], axis=1)
df_v2 = pd.concat([df_v2, pd.get_dummies(df_v1['direction'].astype('category').cat.codes, prefix='direction')], axis=1)
df_v2 = pd.concat([df_v2, pd.get_dummies(df_v1['xydir'].astype('category').cat.codes, prefix='xydir')], axis=1)
df_v2['all'] = df_v1['all']

df_v2_tr = df_v2[df_v2[target_].notnull()]
df_v2_te = df_v2[df_v2[target_].isnull()]

mapper_avg = df_v2_tr[['all',target_]].groupby(['all']).median().to_dict()[target_]
df_v2_tr['avg'] = df_v2_tr['all'].map(mapper_avg)
df_v2_te['avg'] = df_v2_te['all'].map(mapper_avg)

df_v2_tr.drop('all', axis=1, inplace=True)
df_v2_te.drop('all', axis=1, inplace=True)

# df_v0_tr.head()

# df_v2.head()
# df_v2.info()

X = df_v2_tr.drop(target_, axis=1)
y = df_v2_tr[target_]
df_test = df_v2_te.drop(target_, axis=1)

# df_v2_tr.shape, df_v2_te.shape
X_tr.shape, X_te.shape, y_tr.shape

((193700, 92), (2340, 92), (193700,))

In [58]:
from sklearn.metrics import mean_absolute_error as mae
def objective(trial):
    params = {
        "random_state":trial.suggest_categorical("random_state", [519632]),
        'learning_rate' : trial.suggest_loguniform('learning_rate', 0.0001, 0.3),
        'bagging_temperature' :trial.suggest_loguniform('bagging_temperature', 0.01, 100.00),
        "n_estimators": 1000,
        "max_depth":trial.suggest_int("max_depth", 3, 9),
        'random_strength' :trial.suggest_int('random_strength', 0, 100),
        "l2_leaf_reg":trial.suggest_float("l2_leaf_reg",1e-8,3e-5),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        "max_bin": trial.suggest_int("max_bin", 200, 500),
        'od_type': trial.suggest_categorical('od_type', ['IncToDec', 'Iter']),
        'task_type': trial.suggest_categorical('task_type', ['GPU']),
        'loss_function': trial.suggest_categorical('loss_function', ['MAE']),
        'eval_metric': trial.suggest_categorical('eval_metric', ['MAE'])
    }

    model = CatBoostRegressor(**params)
    X_train_tmp, X_valid_tmp, y_train_tmp, y_valid_tmp = train_test_split(X, y, test_size=0.3, random_state=42)
    model.fit(
        X_train_tmp, y_train_tmp,
        eval_set=[(X_valid_tmp, y_valid_tmp)],
        early_stopping_rounds=25, verbose=0
    )
        
    y_train_pred = model.predict(X_train_tmp)
    y_valid_pred = model.predict(X_valid_tmp)
    train_mae = mae(y_train_tmp, y_train_pred)
    valid_mae = mae(y_valid_tmp, y_valid_pred)
    
    print(f'MAE of Train: {train_mae}')
    print(f'MAE of Validation: {valid_mae}')
    
    return valid_mae

In [59]:
allow_optimize = 1

In [62]:
import optuna
from optuna.samplers import TPESampler
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split

TRIALS = 1000
TIMEOUT = 3600 * 6

if allow_optimize:
    sampler = TPESampler(seed=362356)

    study = optuna.create_study(
        study_name = 'cat_parameter_opt',
        direction = 'minimize',
        sampler = sampler,
    )
    study.optimize(objective, n_trials=TRIALS)
    print("Best Score:",study.best_value)
    print("Best trial",study.best_trial.params)
    
    best_params = study.best_params
    
    X_train_tmp, X_valid_tmp, y_train_tmp, y_valid_tmp = train_test_split(X, y, test_size=0.3, random_state=42)
    model_tmp = CatBoostRegressor(**best_params, n_estimators=30000, verbose=1000).fit(X_train_tmp, y_train_tmp, eval_set=[(X_valid_tmp, y_valid_tmp)], early_stopping_rounds=35)

[I 2022-03-27 03:44:42,514] A new study created in memory with name: cat_parameter_opt
[I 2022-03-27 03:44:56,560] Trial 0 finished with value: 5.3595007811346385 and parameters: {'random_state': 519632, 'learning_rate': 0.13637821802218406, 'bagging_temperature': 1.7826226239792804, 'max_depth': 6, 'random_strength': 57, 'l2_leaf_reg': 6.391527496038165e-06, 'min_child_samples': 99, 'max_bin': 266, 'od_type': 'IncToDec', 'task_type': 'GPU', 'loss_function': 'MAE', 'eval_metric': 'MAE'}. Best is trial 0 with value: 5.3595007811346385.


MAE of Train: 5.251877014183133
MAE of Validation: 5.3595007811346385


[I 2022-03-27 03:45:07,321] Trial 1 finished with value: 10.071770574890106 and parameters: {'random_state': 519632, 'learning_rate': 0.0005283375044626603, 'bagging_temperature': 0.7340262520013038, 'max_depth': 5, 'random_strength': 26, 'l2_leaf_reg': 2.65542125669681e-05, 'min_child_samples': 88, 'max_bin': 279, 'od_type': 'Iter', 'task_type': 'GPU', 'loss_function': 'MAE', 'eval_metric': 'MAE'}. Best is trial 0 with value: 5.3595007811346385.


MAE of Train: 10.070992866761705
MAE of Validation: 10.071770574890106


[I 2022-03-27 03:45:09,385] Trial 2 finished with value: 13.91781104801239 and parameters: {'random_state': 519632, 'learning_rate': 0.003301199544227713, 'bagging_temperature': 67.53919280915215, 'max_depth': 5, 'random_strength': 5, 'l2_leaf_reg': 6.082325615930732e-06, 'min_child_samples': 75, 'max_bin': 445, 'od_type': 'Iter', 'task_type': 'GPU', 'loss_function': 'MAE', 'eval_metric': 'MAE'}. Best is trial 0 with value: 5.3595007811346385.


...

MAE of Train: 5.175767504466778
MAE of Validation: 5.345881317977389


[I 2022-03-27 05:06:45,110] Trial 998 finished with value: 5.36045595495514 and parameters: {'random_state': 519632, 'learning_rate': 0.17216896913317797, 'bagging_temperature': 0.6401842322474127, 'max_depth': 9, 'random_strength': 27, 'l2_leaf_reg': 5.871421425597376e-06, 'min_child_samples': 60, 'max_bin': 462, 'od_type': 'IncToDec', 'task_type': 'GPU', 'loss_function': 'MAE', 'eval_metric': 'MAE'}. Best is trial 121 with value: 5.343377942063889.


MAE of Train: 5.183856510520371
MAE of Validation: 5.36045595495514


[I 2022-03-27 05:06:48,943] Trial 999 finished with value: 5.351910986950801 and parameters: {'random_state': 519632, 'learning_rate': 0.10999579435757147, 'bagging_temperature': 1.0316393654724403, 'max_depth': 9, 'random_strength': 26, 'l2_leaf_reg': 6.770563990415716e-06, 'min_child_samples': 62, 'max_bin': 322, 'od_type': 'IncToDec', 'task_type': 'GPU', 'loss_function': 'MAE', 'eval_metric': 'MAE'}. Best is trial 121 with value: 5.343377942063889.


MAE of Train: 5.1776811093001225
MAE of Validation: 5.351910986950801
Best Score: 5.343377942063889
Best trial {'random_state': 519632, 'learning_rate': 0.06613991920995126, 'bagging_temperature': 1.447292201383705, 'max_depth': 9, 'random_strength': 38, 'l2_leaf_reg': 1.1891771199513387e-05, 'min_child_samples': 46, 'max_bin': 481, 'od_type': 'IncToDec', 'task_type': 'GPU', 'loss_function': 'MAE', 'eval_metric': 'MAE'}
0:	learn: 13.2114490	test: 13.2136734	best: 13.2136734 (0)	total: 5.35ms	remaining: 2m 40s
1000:	learn: 5.1556042	test: 5.3433284	best: 5.3433284 (1000)	total: 4.92s	remaining: 2m 22s
bestTest = 5.342686822
bestIteration = 1072
Shrink model to first 1073 iterations.


In [63]:
model = CatBoostRegressor(**best_params, n_estimators=model_tmp.get_best_iteration(), verbose=1000).fit(X, y)

0:	learn: 13.2117114	total: 5.52ms	remaining: 5.91s
1000:	learn: 5.2159974	total: 5.17s	remaining: 368ms
1071:	learn: 5.2033412	total: 5.53s	remaining: 0us


In [68]:
df_sub[target_] = model.predict(df_v2_te).round().copy()

# sep = df_v2_tr[(df_v2_tr['dt_d'] >= 246) & (df_v2_tr['dt_t'] >= 0)]
# lower = sep.groupby(['dt_t', 'x', 'y', 'direction']).congestion.quantile(0.15).values
# upper = sep.groupby(['dt_t', 'x', 'y', 'direction']).congestion.quantile(0.7).values

df_sub[target_] = df_sub[target_].clip(lower, upper)

import datetime

now = datetime.datetime.now()
str_datetime = now.strftime("%y%m%d_%H%M%S")
# df_sub.to_csv(DIR_DATASET+'submission-'+NAME_PROJECT+'-'+str_datetime+'.csv', index = 0)
df_sub.to_csv(DIR_DATASET+'submission-'+NAME_PROJECT+'-'+str_datetime+'.csv')

In [69]:
df_sub.head()

,congestion
row_id,
848835,48.0
848836,34.0
848837,54.0
848838,25.0
848839,73.0


In [ ]:
dir_dataset = "C:/Users/0stix/Datasets/"
name_project = '2203-kaggle-tps2203'
df_train = pd.read_csv(dir_dataset+name_project+'/train.csv')
df_test = pd.read_csv(dir_dataset+name_project+'/test.csv')
df_sub = pd.read_csv(dir_dataset+name_project+'/sample_submission.csv')

len_train = len(df_train)
df_all = pd.concat([df_train, df_test], axis=0)
target = 'congestion'

In [ ]:
cat_reg = CatBoostRegressor(**study_.best_trial.params, n_estimators=model_tmp.get_best_iteration(), verbose=1000)
cat_reg.fit(X_tr, y_tr)
df_sub[target] = cat_reg.predict(X_te)

import datetime
now = datetime.datetime.now()
str_datetime = now.strftime("%y%m%d_%H%M%S")
df_sub.to_csv(dir_dataset+'submission-'+name_project+'-'+str_datetime+'.csv', index = 0)